Classic from 2009. See [GF tutorial](https://www.grammaticalframework.org/doc/tutorial/gf-tutorial.html#toc16) and several concrete syntaxes at [gf-contrib](https://github.com/GrammaticalFramework/gf-contrib/tree/master/foods).

In [1]:
-- (c) 2009 Aarne Ranta under LGPL

abstract Food = {
  flags startcat = Comment ;
  cat
    Comment ;  -- Start category
    Item ; 
    Kind ;     -- "nouns"
    Quality ;  -- "adjectives"
  fun
    Pred : 
      Item -> Quality -> Comment ;       -- Predication: "item is quality"
    
    
    This, That, These, Those :
      Kind -> Item ; -- Quantification: turn the Kind ("pizza") into Item ("this pizza")
    
    Mod : 
     Quality -> Kind -> Kind ;  -- Modification: "pizza" to "boring pizza"

    Wine, Cheese, Fish, Pizza : Kind ;        -- list of Kinds
    
    Very : Quality -> Quality ;               -- Enhance a Quality: "boring" to "very boring"
    Fresh, Warm, Italian, Boring,
         Expensive, Delicious : Quality ;     -- list of Qualities
}

Defined Food

In [2]:
gr

Pred (These Cheese) Italian


In [4]:
concrete FoodEng of Food = {
    lincat
      Comment = {s : Str} ;
      Quality = {
          s : Str ;
          pq : PQual
      } ;

      Item = {s : Str ; n : Number} ;  -- Inherent Number
    
      Kind = {s : Number => Str ;
              hasBoring, hasItalian : Bool
             } ;     -- Variable Number

    param
      Number = Sg | Pl ;
      Bool = True | False ;
      PQual = PBoring | PItalian ;
    
    lin
      {- : Item ->     (argument 1)
          Quality ->    (argument 2)
         Comment       (result / return type) -}
      Pred item qual = {
          s = case item.n of {
              Sg => item.s ++ "is" ++ qual.s ;
              Pl => item.s ++ "are" ++ qual.s }
      } ;
    
      -- : Kind -> Item
      This kind = {
          s = "this" ++ kind.s ! Sg ;
          n = Sg
      } ;
      Those kind = {
          s = "those" ++ kind.s ! Pl ;
          n = Pl
      } ;
    
      -- : Quality -> Kind -> Kind
      Mod qual kind = case qual.pq of {
          PBoring => case kind.hasBoring of {
                  True => kind ;
                  False => kind ** {
                      s = \\n => qual.s ++ kind.s ! n ;
                      hasBoring = True }
          } ;
          PItalian => case kind.hasItalian of {
                  True => kind ;
                  False => kind ** {
                      s = \\n => qual.s ++ kind.s ! n ;
                      hasItalian = True} 
          } 
      } ;  
    
      Boring = {s = "boring" ; pq = PBoring} ;
      Italian = {s = "Italian" ; pq = PItalian} ;  -- : Quality
    
      -- : Kind
      Pizza = {s =
                table {
                  Sg => "pizza" ;
                  Pl => "pizzas"
                 } ;
               hasBoring, hasItalian = False ;
              } ;
    
    oper
    
      someFun : Str -> Str = \_ -> "always hello" ;
          
      someTable : Number => Str = 
          \\n => "always singular" ;
          -- shorthand for table {n => "always singular"} ;
          -- shorthand for table {Sg => "always singular ; Pl => "always singular"} ;
}

Defined FoodEng

In [7]:
gt | l -treebank

Food: Pred (This (Mod Boring (Mod Boring Pizza))) Boring
FoodEng: this boring pizza is boring
Food: Pred (This (Mod Boring (Mod Boring Pizza))) Italian
FoodEng: this boring pizza is Italian
Food: Pred (This (Mod Boring (Mod Italian Pizza))) Boring
FoodEng: this boring Italian pizza is boring
Food: Pred (This (Mod Boring (Mod Italian Pizza))) Italian
FoodEng: this boring Italian pizza is Italian
Food: Pred (This (Mod Boring Pizza)) Boring
FoodEng: this boring pizza is boring
Food: Pred (This (Mod Boring Pizza)) Italian
FoodEng: this boring pizza is Italian
Food: Pred (This (Mod Italian (Mod Boring Pizza))) Boring
FoodEng: this Italian boring pizza is boring
Food: Pred (This (Mod Italian (Mod Boring Pizza))) Italian
FoodEng: this Italian boring pizza is Italian
Food: Pred (This (Mod Italian (Mod Italian Pizza))) Boring
FoodEng: this Italian pizza is boring
Food: Pred (This (Mod Italian (Mod Italian Pizza))) Italian
FoodEng: this Italian pizza is Italian
Food: Pred (This (Mod Italian Pizz

### English concrete syntax from scratch

We define our own inflection tables and agreement rules.

In [ ]:
concrete FoodEng of Food = {
  
    lincat
      Comment, Quality = {s : Str} ;
      Kind = Noun ;       -- Noun defined later in this file.
      Item = NounPhrase ; -- NounPhrase defined later in this file.
  
    lin
      -- Syntactic functions that take arguments
      Pred item quality = {s = item.s ++ copula item.n ++ quality.s} ;
      This kind  = det Sg "this" kind ;
      That kind  = det Sg "that" kind ;
      These kind = det Pl "these" kind ;
      Those kind = det Pl "those" kind ;
      Mod quality kind = {s = table {n => quality.s ++ kind.s ! n}} ;

      -- Lexical functions, no arguments
      Wine   = regNoun "wine" ;
      Cheese = regNoun "cheese" ;
      Fish   = noun "fish" "fish" ;
      Pizza  = regNoun "pizza" ;
      Very qual = {s = "very" ++ qual.s} ;
      Fresh     = adj "fresh" ;
      Warm      = adj "warm" ;
      Italian   = adj "Italian" ;
      Expensive = adj "expensive" ;
      Delicious = adj "delicious" ;
      Boring    = adj "boring" ;

    param
      Number = Sg | Pl ;
  
    oper
      -- Defining our own record types.
      -- They are used in the lincats.
      Noun : Type = {s : Number => Str} ;
      NounPhrase : Type = {s : Str ; n : Number} ;
    
      -- Makes Noun into a Noun phrase.
      det : Number -> Str -> Noun -> NounPhrase =
        \n,d,cn -> {
          s = d ++ cn.s ! n ;
          n = n
        } ;
    
      -- Constructs a noun from singular and plural.
      noun : Str -> Str -> Noun =
        \man,men -> {s = table {
          Sg => man ;
          Pl => men
          }
        } ;
      -- Regular noun: only takes singular, adds "s" to form plural.
      regNoun : Str -> Noun =
        \car -> noun car (car + "s") ;
    
      -- Adjective is just a single string.
      adj : Str -> {s : Str} =
        \red -> {s = red} ;
  
      -- Copula inflects in number.
      copula : Number -> Str =
        \n -> case n of {
          Sg => "is" ;
          Pl => "are"
          } ;
    }

### Italian concrete syntax using RGL modules

We use the Resource Grammar Library to hide the linguistic details.

See also Tutorial _[Lesson 4: Using the resource grammar library](https://www.grammaticalframework.org/doc/tutorial/gf-tutorial.html#toc69)_

In [ ]:
concrete FoodIta of Food = open SyntaxIta, ParadigmsIta in {
    lincat
      Comment = S ;
      Item = NP ;
      Kind = CN ;
      Quality = AP ;

    lin
      Pred item quality = mkS (mkCl item quality) ;
      This kind = mkNP this_Det kind ;
      That kind = mkNP that_Det kind ;
      These kind = mkNP these_Det kind ;
      Those kind = mkNP those_Det kind ;
      Mod quality kind = mkCN quality kind ;
      Very quality = mkAP very_AdA quality ;
      Wine = mkCN (mkN "vino") ;
      Pizza = mkCN (mkN "pizza") ;
      Cheese = mkCN (mkN "formaggio") ;
      Fish = mkCN (mkN "pesce") ;
      Fresh = mkAP (mkA "fresco") ;
      Warm = mkAP (mkA "caldo") ;
      Italian = mkAP (mkA "italiano") ;
      Expensive = mkAP (mkA "caro") ;
      Delicious = mkAP (mkA "delizioso") ;
      Boring = mkAP (mkA "noioso") ;
}

In [ ]:
parse -lang=Eng "this pizza is boring" | linearize -lang=Ita

In [ ]:
view parse -lang=Eng "this Italian pizza is very expensive" | visualize_parse -lang=Eng

In [ ]:
view p -lang=Ita "questa pizza italiana è molto cara" | vp -lang=Ita